# Cell 1: Setup and Master Configuration
**This cell handles all installations, imports, model downloads, and contains a single configuration section for all your settings.**

In [ ]:
# For local environments (like Anaconda), uncomment these lines to install libraries.
# !pip install pandas openpyxl transformers torch tqdm textblob seaborn nltk scikit-learn
# !pip install --upgrade transformers huggingface_hub
# !pip install --upgrade Pillow==9.5.0
!pip install transformers==4.35.2 accelerate==0.24.1 huggingface_hub==0.19.4
!pip install langdetect
!pip install deep-translator
!pip install ipywidgets

# --- Import Libraries ---
import pandas as pd
import os
import re
import string
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import scipy.stats as stats
import seaborn as sns
import nltk
import spacy
from transformers import pipeline
from tqdm.auto import tqdm
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import FreqDist, bigrams, trigrams
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import silhouette_score
from openpyxl.drawing.image import Image
from openpyxl.styles import Alignment, Border, Side
from gensim.corpora.dictionary import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

# --- Input desired plot style here ---
plt.style.use('seaborn-v0_8-deep') 

# --- Download NLP Models ---
# This section ensures all necessary data models are available.
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('vader_lexicon', quiet=True)
nltk.download('omw-1.4', quiet=True)
nltk.download('brown', quiet=True)
!python -m textblob.download_corpora --quiet
!python -m spacy download en_core_web_md --quiet

# Register tqdm for use with pandas .progress_apply()
tqdm.pandas(desc="Processing Verbatims")

# ==============================================================================
#  2. 🔴 MASTER USER CONFIGURATION 🔴
# ==============================================================================

# --- Input File Details ---
# For Kaggle, the path is typically '/kaggle/input/your-dataset-name/your-file-name.xlsx'
FILE_PATH = r'/kaggle/input/fanatics-verbatim-jun-aug-2025-02/Fanatics Verbatim Data June-August 2025.xlsx'
TEXT_COLUMN_NAME = 'Survey Verbatim'

# --- Analysis Details ---
OUTPUT_FOLDER_PATH = r'/kaggle/working/' # Correct for Kaggle
PROGRAM_NAME = "Fanatics"
KPIS_IN_SCOPE = "CSAT"
LOBS_IN_SCOPE = "Voice and Chat"
SCORE_COLUMN_NAME = 'CSAT Score'
MAJOR_VERSION = 1 # Manually change for new datasets or category versions

# --- Analysis Settings ---
CUSTOM_STOP_WORDS = {
    # --- Generic Business & Service Terms ---
    # 'company', 'business', 'service', 'services', 'product', 'products', 'team', 'customer', 
    # 'customers', 'client', 'clients', 'staff', 'agent', 'representative', 'organization',
    # 'experience', 'issue', 'issues', 'problem', 'problems', 'question', 'questions', 
    # 'feedback', 'inquiry', 'inquiries', 'request', 'requests', 'ticket', 'case', 'account',

    # --- Common Vague Fillers & Conversational Terms ---
    'also', 'really', 'actually', 'just', 'like', 'im', 'ive', 'thing', 'things', 
    'something', 'anything', 'everything', 'well', 'get', 'got', 'getting', 'would', 'could', 
    'should', 'make', 'made',
    
    # 'one', 'even', 'since', 'every', 'time', 'times', 'day', 'days', 
    # 'week', 'weeks', 'month', 'months', 'year', 'years', 'lot', 'always'

    # --- Politeness, Greetings, & Inquiries ---
    # 'please', 'help', 'hello', 'hi', 'hey', 'thank', 'thanks', 'appreciate', 'regards', 'best', 'since', 'under', 'within',
    # 'know', 'see', 'want', 'wanted', 'looking', 'wondering', 'information', 'info', 'details', 'beyond', 'over',
    
    # --- Placeholders for Your Specific Names ---
    'company',str(PROGRAM_NAME),str(KPIS_IN_SCOPE),str(LOBS_IN_SCOPE),

    # --- Prepositions ---
    'aboard', 'about', 'above', 'across', 'after', 'against', 'along', 'among', 'around', 'at',
    'before', 'behind', 'below', 'beneath', 'beside', 'between', 'by', 'down', 'during',
    'for', 'from', 'in', 'inside', 'into', 'like', 'near', 'of', 'off', 'on', 'onto', 'out',
    'outside', 'past', 'through', 'throughout', 'to', 'toward',
    'underneath', 'until', 'up', 'upon', 'with', 'without', 'is', 'a', 'or'
}

CLASSIFICATION_THRESHOLD = 0.55 # Confidence score (0.0 to 1.0) for categorization

# --- Define Your Zero-Shot Categories and Sub-Categories (Granular Version) ---
CATEGORIES = {
    # --- People Driven Categories ---
    'Interaction with Call Center Agent': [
        "Call agent's communication and listening skills",
        "Call agent's knowledge and problem-solving ability",
        "Call agent's attitude empathy and professionalism",
        "Efficiency and speed of call handling or resolution",
    ],
    'Interaction with In-Store Staff': [
        "In-store staff's helpfulness and attitude",
        "Staff's product knowledge and ability to answer questions",
        "Availability and attentiveness of staff in the store",
        "Efficiency of in-store processes like checkout or returns",
    ],
    'Interaction with Field Technician': [
        "Technician's professionalism, timeliness, and communication",
        "Technician's skill and ability to fix the issue",
        "Cleanliness and care taken by the technician in the home",
        "Explanation of work performed by the technician",
    ],

    # --- Process Driven Category ---
    'Company Process or Policy Issue': [
        'Confusion or disagreement with a company policy',
        'The overall process was too complex or had too many steps',
        'The total time it took to resolve the issue',
        'Problems with a follow-up, callback, or promised contact',
    ],

    # --- Technical and System Categories ---
    'Website or Online Portal Issue': [
        "Website was slow, lagging, or unresponsive",
        'Difficulty navigating or finding information on the website',
        'A website bug, glitch, or error message',
        'The website crashed, froze, or was unavailable',
    ],
    'Mobile Application Issue': [
        'The mobile app was slow or had poor performance',
        'A bug or error in the mobile app',
        'The mobile app crashed or froze',
        'The mobile app was difficult to use or understand',
    ],
    'Communication Channel Quality': [
        'Poor audio quality, static, or bad phone connection',
        'Loud background noise during a call',
        'Issues with the live chat tool or functionality',
        'Problems with email communication or response times',
    ],

    # --- Product Driven Category ---
    'Feedback on the Product Itself': [
        'The quality, a defect, or damage of the product',
        'A suggestion or request for a new product feature',
        'Feedback on the price, cost, or value for money',
        'The design, appearance, or general ease of use of the product',
    ]
}

print("✅ Setup complete. All libraries and models are ready.")

# Cell 1.1: Category Selector

In [ ]:
# Import the ipywidgets library
import ipywidgets as widgets
from IPython.display import display

# Get your list of Level 1 categories from the CATEGORIES dictionary
all_categories = list(CATEGORIES.keys())

# Create the interactive multi-select widget
category_selector = widgets.SelectMultiple(
    options=all_categories,
    value=all_categories, # Pre-selects all categories by default
    description='Categories:',
    disabled=False,
    layout={'width': '50%'} # Adjust width as needed
)

# Display the widget in the cell's output
print("Hold Ctrl (or Cmd on Mac) to select multiple categories.")
display(category_selector)

# Cell 2: Data Loading, Cleaning, and Sentiment Analysis
**This cell prepares your core DataFrame by loading, cleaning, and running sentiment analysis.**

In [ ]:
# --- 1. Load Data Safely ---
try:
    df = pd.read_excel(FILE_PATH)
    print(f"✅ Successfully loaded {len(df)} rows from '{FILE_PATH}'.")

    # --- 2. Text Cleaning (UPGRADED AND FINAL) ---
    # Import necessary libraries
    from textblob import TextBlob
    from langdetect import detect, LangDetectException
    from deep_translator import GoogleTranslator # ✨ USE DEDICATED TRANSLATOR

    stop_words = set(stopwords.words('english')).union(CUSTOM_STOP_WORDS)
    lemmatizer = WordNetLemmatizer()

    def clean_text(text):
        if not isinstance(text, str) or len(text.strip()) < 10:
            return ""

        # ✨ STEP 1: Detect language and translate
        try:
            lang = detect(text)
            if lang != 'en':
                # Use the new, more reliable translator
                text = GoogleTranslator(source='auto', target='en').translate(text)
        except Exception as e:
            # If detection or translation fails, just proceed with the original text
            pass

        # ✨ STEP 2: Correct spelling (using TextBlob for this part is generally fine)
        text = str(TextBlob(text).correct())

        # --- Original cleaning steps ---
        text = text.lower()
        text = re.sub(r'[\d\n]', '', text)
        text = text.translate(str.maketrans('', '', string.punctuation))
        tokens = word_tokenize(text.strip())
        cleaned_tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stop_words and len(w) > 2]
        return " ".join(cleaned_tokens)

    df.dropna(subset=[TEXT_COLUMN_NAME], inplace=True)
    df['cleaned_text'] = df[TEXT_COLUMN_NAME].progress_apply(clean_text)

    # --- 3. Sentiment Analysis ---
    sia = SentimentIntensityAnalyzer()
    df['sentiment_compound'] = df[TEXT_COLUMN_NAME].apply(lambda x: sia.polarity_scores(x)['compound'])
    def categorize_sentiment(compound):
        if compound >= 0.05: return 'Positive'
        if compound <= -0.05: return 'Negative'
        return 'Neutral'
    df['sentiment_label'] = df['sentiment_compound'].apply(categorize_sentiment)

    print("\n--- Data Preview with Cleaned Text and Sentiment ---")
    display(df[[TEXT_COLUMN_NAME, 'cleaned_text', 'sentiment_label']].head())
    
# --- 4. Plot Sentiment Distribution and Save ---
    
    sentiment_counts = df['sentiment_label'].value_counts()
    
    plt.style.use('seaborn-v0_8-deep') 
    plt.figure(figsize=(6, 4))
    
    # Plot the data using the new variable
    sentiment_counts.plot(kind='bar', color=['green', 'red', 'grey'], width=0.9)
    
    # ADDED: Loop through the counts to add text labels (call outs)
    for i, count in enumerate(sentiment_counts):
        # The plt.text() function places text on the plot
        # x=i: the horizontal position of the bar (0, 1, 2...)
        # y=count: the vertical position (the height of the bar)
        # s=str(count): the text to display (the count as a string)
        # ha='center': horizontally center the text on the bar
        plt.text(i, count + (sentiment_counts.max() * 0.01), str(count), ha='center')

    plt.title('Sentiment Distribution')
    plt.ylabel('Number of Responses')
    plt.xticks(rotation=0)
    
    # Adjust y-axis to make space for the labels
    plt.ylim(0, sentiment_counts.max() * 1.1) 

    fig = plt.gcf()
    fig.patch.set_edgecolor('grey')
    fig.patch.set_linewidth(1)
    
    plt.savefig(os.path.join(OUTPUT_FOLDER_PATH, 'sentiment_distribution.png'), bbox_inches='tight')
    plt.show()

except FileNotFoundError:
    print("="*80)
    print(f"❌ FATAL ERROR: File not found at the specified path.")
    print(f"    Your specified path: '{FILE_PATH}'")
    print("    Please check the 'FILE_PATH' variable in your configuration cell and try again.")
    print("="*80)
    raise

# Cell 3: Topic Evaluation (Silhouette & Coherence)

In [ ]:
# --- 1. Prepare Data for Evaluation ---
eval_vectorizer = CountVectorizer(max_df=0.9, min_df=5, stop_words='english')
dtm_eval = eval_vectorizer.fit_transform(df['cleaned_text'].dropna())
texts_for_gensim = [word_tokenize(text) for text in df['cleaned_text'].dropna()]
dictionary = Dictionary(texts_for_gensim)
corpus = [dictionary.doc2bow(text) for text in texts_for_gensim]

# --- 2. Define a Range of Topics to Test ---
min_topics = 2
max_topics = 11
topic_range = range(min_topics, max_topics)

# --- 3. Calculate Scores for Each Number of Topics ---
silhouette_scores = []
coherence_scores = []
print("Evaluating optimal number of topics (k). This may take several minutes...")
for k in topic_range:
    # Silhouette Score
    lda_sklearn = LatentDirichletAllocation(n_components=k, random_state=42)
    lda_sklearn.fit(dtm_eval)
    if len(np.unique(lda_sklearn.transform(dtm_eval).argmax(axis=1))) > 1:
        score = silhouette_score(dtm_eval, lda_sklearn.transform(dtm_eval).argmax(axis=1))
        silhouette_scores.append(score)
    else:
        silhouette_scores.append(-1)
    
    # Coherence Score
    lda_gensim = LdaModel(corpus=corpus, id2word=dictionary, num_topics=k, random_state=42)
    coherence_model = CoherenceModel(model=lda_gensim, texts=texts_for_gensim, dictionary=dictionary, coherence='c_v')
    coherence = coherence_model.get_coherence()
    coherence_scores.append(coherence)
    print(f"  - Processed k={k} topics...")

# --- 4. Find the Optimal Number of Topics ---
# Handle cases with no variance in scores to prevent errors
s_range = np.max(silhouette_scores) - np.min(silhouette_scores)
c_range = np.max(coherence_scores) - np.min(coherence_scores)
norm_silhouette = (silhouette_scores - np.min(silhouette_scores)) / s_range if s_range > 0 else np.zeros(len(silhouette_scores))
norm_coherence = (coherence_scores - np.min(coherence_scores)) / c_range if c_range > 0 else np.zeros(len(coherence_scores))
combined_score = norm_silhouette + norm_coherence
best_k_index = np.argmax(combined_score)
recommended_k = topic_range[best_k_index]

# --- 5. Plot and Save the Results ---
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
# Plot Silhouette
ax1.plot(topic_range, silhouette_scores, marker='o', color='b')
ax1.set_title('Silhouette Score vs. Number of Topics')
ax1.axvline(x=recommended_k, color='grey', linestyle='--', label=f'Recommended k={recommended_k}')
ax1.legend()
# Plot Coherence
ax2.plot(topic_range, coherence_scores, marker='o', color='r')
ax2.set_title('Topic Coherence (C_v) vs. Number of Topics')
ax2.axvline(x=recommended_k, color='grey', linestyle='--', label=f'Recommended k={recommended_k}')
ax2.legend()
plt.tight_layout()

fig = plt.gcf()
fig.patch.set_edgecolor('grey')
fig.patch.set_linewidth(1)

plt.savefig(os.path.join(OUTPUT_FOLDER_PATH, 'topic_evaluation_charts.png'), bbox_inches='tight')
plt.show()

# --- 6. Store Results and Update Variable ---
evaluation_results = {
    'Num_Topics (k)': list(topic_range),
    'Silhouette_Score': silhouette_scores,
    'Coherence_Score': coherence_scores
}
df_topic_evaluation = pd.DataFrame(evaluation_results)

# --- 7. Print Summary and Update TOPIC_MODEL_TOPICS Variable ---
print("\n--- Evaluation Results Table ---")
display(df_topic_evaluation.round(3))

print("\n--- Automated Recommendation ---")
print(f"Best Silhouette Score at k = {topic_range[np.argmax(silhouette_scores)]}")
print(f"Best Coherence Score at k = {topic_range[np.argmax(coherence_scores)]}")
print("-" * 30)
print(f"🏆 Recommended number of topics (best combined score): {recommended_k}")
print("-" * 30)

TOPIC_MODEL_TOPICS = recommended_k
print(f"✅ The 'TOPIC_MODEL_TOPICS' variable has been automatically set to {TOPIC_MODEL_TOPICS}.")

# Cell 4: Exploratory Analysis (Frequency, N-grams, Word Clouds)
**This cell prepares frequency tables and word cloud images for the final report.**

In [ ]:
# --- 1. Prepare Overall Text and Tokens ---
all_cleaned_text = " ".join(df['cleaned_text'])
all_tokens = word_tokenize(all_cleaned_text)

# --- 2. Create and Store Frequency Tables ---
fdist = FreqDist(all_tokens)
df_top_words = pd.DataFrame(fdist.most_common(20), columns=['Word', 'Frequency'])

bigram_fdist = FreqDist(list(bigrams(all_tokens)))
# CORRECTED: Provide both the bigram and its frequency to the DataFrame
df_top_bigrams = pd.DataFrame([(' '.join(gram), freq) for gram, freq in bigram_fdist.most_common(10)], columns=['Bigram', 'Frequency'])

trigram_fdist = FreqDist(list(trigrams(all_tokens)))
# CORRECTED: Provide both the trigram and its frequency to the DataFrame
df_top_trigrams = pd.DataFrame([(' '.join(gram), freq) for gram, freq in trigram_fdist.most_common(10)], columns=['Trigram', 'Frequency'])

print("--- Top 20 Most Common Words ---")
display(df_top_words)

print("--- Top 10 Most Common Bigrams ---")
display(df_top_bigrams)

print("--- Top 10 Most Common Trigrams ---")
display(df_top_trigrams)

# --- 3. Generate and Save Word Clouds ---
def generate_and_save_wordcloud(text, title, filename):
    if not text.strip():
        print(f"Skipping '{title}' word cloud: No text available.")
        return
    wordcloud = WordCloud(width=800, height=400, background_color='white', colormap='viridis').generate(text)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(title)

    fig = plt.gcf()
    fig.patch.set_edgecolor('grey')
    fig.patch.set_linewidth(1)
    
    plt.savefig(os.path.join(OUTPUT_FOLDER_PATH, filename), bbox_inches='tight')
    plt.show()

generate_and_save_wordcloud(all_cleaned_text, 'Word Cloud (All Feedback)', 'wordcloud_all.png')
generate_and_save_wordcloud(" ".join(df[df.sentiment_label == 'Positive']['cleaned_text']), 'Word Cloud (Positive)', 'wordcloud_positive.png')
generate_and_save_wordcloud(" ".join(df[df.sentiment_label == 'Negative']['cleaned_text']), 'Word Cloud (Negative)', 'wordcloud_negative.png')

# Cell 5: Thematic Analysis (Topic Modeling and Zero-Shot Categorization)
**This cell performs the main "what are they talking about?" analyses.**

In [ ]:
# # --- 1. Topic Modeling (LDA) ---
# print("\n--- Discovering Latent Topics (LDA) ---")
# vectorizer = CountVectorizer(max_df=0.9, min_df=3, stop_words='english')
# dtm = vectorizer.fit_transform(df['cleaned_text'].dropna())
# if dtm.shape[0] > 1 and dtm.shape[1] > 1:
#     lda = LatentDirichletAllocation(n_components=TOPIC_MODEL_TOPICS, random_state=42)
#     lda.fit(dtm)
#     topic_results = []
#     feature_names = vectorizer.get_feature_names_out()
#     for topic_idx, topic in enumerate(lda.components_):
#         top_words_str = ", ".join([feature_names[i] for i in topic.argsort()[:-10 - 1:-1]])
#         topic_results.append([f"Topic #{topic_idx + 1}", top_words_str])
#     df_topics = pd.DataFrame(topic_results, columns=['Discovered Topic', 'Top Words'])
#     display(df_topics)
# else:
#     print("Not enough data to perform topic modeling.")
#     df_topics = pd.DataFrame() # Create empty df if it fails

# # --- 2. Zero-Shot Root Cause Categorization ---
# print("\n--- Loading Zero-Shot Classification model ---")
# classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli",device=0)

# def get_multi_label_predictions(text, labels, threshold):
#     if not text or not isinstance(text, str): return []
#     results = classifier(text, candidate_labels=labels, multi_label=True)
#     return [label for i, label in enumerate(results['labels']) if results['scores'][i] >= threshold]

# def extract_key_phrases(text):
#     return "|".join([str(p) for p in TextBlob(text).noun_phrases[:3]]) if text else ""

# def categorize_row(row):
#     text = row[TEXT_COLUMN_NAME]
#     matched_cats = get_multi_label_predictions(text, list(CATEGORIES.keys()), CLASSIFICATION_THRESHOLD)
#     matched_subcats = []
#     if matched_cats:
#         for cat in matched_cats:
#             sub_preds = get_multi_label_predictions(text, CATEGORIES.get(cat, []), CLASSIFICATION_THRESHOLD)
#             matched_subcats.extend(sub_preds)
#         if not matched_subcats and (key_phrases := extract_key_phrases(text)):
#             matched_subcats.append(f"SUGGESTION: {key_phrases}")
#     return "|".join(matched_cats) if matched_cats else 'Uncategorized', "|".join(matched_subcats) if matched_subcats else ""

# print(f"\n--- Starting Zero-Shot categorization with a threshold of {CLASSIFICATION_THRESHOLD:.2f} ---")
# df[['Category', 'Sub-Category']] = df.progress_apply(categorize_row, axis=1, result_type='expand')

# print("\n--- Categorization Complete ---")

# Revised 5.0.1

# Cell 6: Deep-Dive Categorization Analysis
**This cell is dedicated to analyzing the results of our categorization, creating the tables and charts needed for the executive summary.**

In [ ]:
# --- 1. Create Exploded DataFrames (for re-use) ---
df_exploded_cat = df.assign(Category=df['Category'].str.split('|')).explode('Category')

df_exploded_subcat = df.copy()
df_exploded_subcat['Sub-Category'] = df_exploded_subcat['Sub-Category'].str.split('|')
df_exploded_subcat = df_exploded_subcat.explode('Sub-Category')

df_exploded_subcat.dropna(subset=['Sub-Category'], inplace=True)
df_exploded_subcat = df_exploded_subcat[df_exploded_subcat['Sub-Category'].str.strip() != '']

# --- 2. Categorization Summary ---
total_rows = len(df)
uncategorized_count = len(df[df['Category'] == 'Uncategorized'])
categorized_count = total_rows - uncategorized_count
categorization_rate = (categorized_count / total_rows) * 100 if total_rows > 0 else 0
print("\n--- Categorization Summary ---")
summary_metrics = {
    'Metric': ['Total Verbatims', 'Categorized', 'Uncategorized', 'Categorization Rate'],
    'Value': [total_rows, categorized_count, uncategorized_count, f"{categorization_rate:.2f}%"]
}
df_summary_metrics = pd.DataFrame(summary_metrics)
display(df_summary_metrics)

# --- 3. Sentiment Breakdown Tables ---
print("\n--- Sentiment Breakdown by Category ---")
df_sentiment_by_cat = pd.crosstab(df_exploded_cat['Category'], df_exploded_cat['sentiment_label'])
df_sentiment_by_cat['Total'] = df_sentiment_by_cat.sum(axis=1)
df_sentiment_by_cat.sort_values(by='Total', ascending=False, inplace=True)
display(df_sentiment_by_cat.drop(columns='Total'))

print("\n--- Sentiment Breakdown by Sub-Category ---")
df_sentiment_by_subcat = pd.crosstab(df_exploded_subcat['Sub-Category'], df_exploded_subcat['sentiment_label'])
df_sentiment_by_subcat['Total'] = df_sentiment_by_subcat.sum(axis=1)
df_sentiment_by_subcat.sort_values(by='Total', ascending=False, inplace=True)
display(df_sentiment_by_subcat.drop(columns='Total'))

# --- 4. Top Keywords for "Uncategorized" Verbatims ---
print("\n--- Top Keywords in Uncategorized Verbatims ---")
uncategorized_text = " ".join(df[df['Category'] == 'Uncategorized']['cleaned_text'])
if uncategorized_text.strip():
    uncategorized_fdist = FreqDist(word_tokenize(uncategorized_text))
    df_uncategorized_keywords = pd.DataFrame(uncategorized_fdist.most_common(20), columns=['Keyword', 'Frequency'])
    display(df_uncategorized_keywords.head(10))
else:
    print("No keywords to display for uncategorized verbatims.")
    df_uncategorized_keywords = pd.DataFrame(columns=['Keyword', 'Frequency'])

# --- 5. Category and Sub-Category Frequency Analysis & Visualization (UPDATED) ---

# ✨ UPDATED function to accept total_verbatims and plot percentages
def plot_and_save_top_n(series, title, filename, total_verbatims, n=15):
    """Helper function to plot and save frequency charts as percentages."""
    if series.empty or total_verbatims == 0:
        print(f"Skipping plot '{title}': No data.")
        return
        
    # Convert counts to percentage
    series_pct = series.head(n).sort_values(ascending=True) / total_verbatims
    
    plt.figure(figsize=(12, 8))
    ax = series_pct.plot(kind='barh', color='darkslateblue', width=0.9)
    
    # Format the x-axis as percentages
    ax.xaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    
    # --- ADDED: Loop to add data call outs ---
    for i, value in enumerate(series_pct):
        # ax.text() places text on the axes
        # x-coordinate: value (end of the bar) + a small offset
        # y-coordinate: i (the position of the bar, 0, 1, 2...)
        # text: formatted as a percentage string (e.g., '15.2%')
        # va='center': vertically aligns the text with the middle of the bar
        ax.text(value + (series_pct.max() * 0.01), i, f'{value:.1%}', va='center')

    plt.title(title)
    plt.xlabel('Percentage of Total Verbatims (%)') # Updated label
    
    # Adjust x-axis limit to make space for the labels
    ax.set_xlim(right=series_pct.max() * 1.15)
    
    plt.subplots_adjust(left=0.4)
    
    fig = plt.gcf()
    fig.patch.set_edgecolor('grey')
    fig.patch.set_linewidth(1)

    plt.savefig(os.path.join(OUTPUT_FOLDER_PATH, filename), bbox_inches='tight')
    plt.show()

# --- Calculate Overall Frequencies ---
print("\n--- Calculating and Plotting Overall Frequencies ---")
total_verbatims = len(df) # Get total for calculations
df_cat_counts = df_exploded_cat['Category'].value_counts()
df_subcat_counts = df_exploded_subcat['Sub-Category'].value_counts()

# ✨ UPDATED function calls to pass total_verbatims
plot_and_save_top_n(df_cat_counts.drop('Uncategorized', errors='ignore'), 'Overall Top Categories', 'freq_cat_overall.png', total_verbatims)
plot_and_save_top_n(df_subcat_counts, 'Overall Top Sub-Categories', 'freq_subcat_overall.png', total_verbatims)

# --- Calculate Frequencies Split by Sentiment ---
print("\n--- Calculating and Plotting Frequencies by Sentiment ---")
for sentiment in ['Positive', 'Negative', 'Neutral']:
    df_sentiment_exploded_cat = df_exploded_cat[df_exploded_cat['sentiment_label'] == sentiment]
    if not df_sentiment_exploded_cat.empty:
        cat_counts = df_sentiment_exploded_cat['Category'].value_counts()
        # ✨ UPDATED function calls
        plot_and_save_top_n(cat_counts, f'Top Categories ({sentiment} Sentiment)', f'freq_cat_{sentiment.lower()}.png', total_verbatims)

    df_sentiment_exploded_subcat = df_exploded_subcat[df_exploded_subcat['sentiment_label'] == sentiment]
    if not df_sentiment_exploded_subcat.empty:
        subcat_counts = df_sentiment_exploded_subcat['Sub-Category'].value_counts()
        # ✨ UPDATED function calls
        plot_and_save_top_n(subcat_counts, f'Top Sub-Categories ({sentiment} Sentiment)', f'freq_subcat_{sentiment.lower()}.png', total_verbatims)

# Cell 7: Breakdown of Sub-categories by Sentiment and Category

In [ ]:
print("--- 📊 Generating Validated Breakdown, Plots, & Examples ---")

# --- [Keep the sanitize_filename helper function as is] ---
def sanitize_filename(name):
    name = re.sub(r'[^\w\s-]', '', name).strip().lower()
    name = re.sub(r'[-\s]+', '_', name)
    return name

# ✨ NEW: Get the total number of verbatims for percentage calculations
total_verbatims = len(df)
print(f"Total verbatims being processed for percentages: {total_verbatims}")

# --- [Keep the data preparation steps as they were] ---
valid_pairs = set()
for category, subcategories in CATEGORIES.items():
    for subcategory in subcategories:
        valid_pairs.add((category, subcategory))

df_exploded_all_links = df.assign(Category=df['Category'].str.split('|')).explode('Category')
df_exploded_all_links = df_exploded_all_links.assign(SubCategory=df_exploded_all_links['Sub-Category'].str.split('|')).explode('SubCategory')
df_exploded_all_links.dropna(subset=['Category', 'SubCategory'], inplace=True)
df_exploded_all_links = df_exploded_all_links[
    (df_exploded_all_links['Category'].str.strip() != '') &
    (df_exploded_all_links['SubCategory'].str.strip() != '')
]
df_validated_links = df_exploded_all_links[
    df_exploded_all_links.apply(lambda row: (row['Category'], row['SubCategory']) in valid_pairs, axis=1)
]

# --- Perform crosstabulation and display results ---
if not df_validated_links.empty:
    granular_sentiment_df = pd.crosstab(
        [df_validated_links['Category'], df_validated_links['SubCategory']],
        df_validated_links['sentiment_label']
    )
    for sentiment in ['Negative', 'Neutral', 'Positive']:
        if sentiment not in granular_sentiment_df.columns:
            granular_sentiment_df[sentiment] = 0
    granular_sentiment_df['Total'] = granular_sentiment_df.sum(axis=1)
    
    unique_categories = granular_sentiment_df.index.get_level_values('Category').unique()

    for category in unique_categories:
        print("\n" + "#"*80)
        print(f"## CATEGORY: {category}")
        print("#"*80)
        
        category_view = granular_sentiment_df.loc[category].sort_values(by='Total', ascending=False)
        category_view = category_view[['Negative', 'Neutral', 'Positive', 'Total']]
        display(category_view)
        
        # --- ✨ NEW: VERBATIM EXAMPLES SECTION ---
        print("\n" + "-"*40)
        print("Verbatim Examples:")
        print("-"*40)
        # Loop through the sub-categories in the table we just displayed
        for sub_cat in category_view.index:
            # Find the original verbatims that match this exact category and sub-category
            examples = df_validated_links[
                (df_validated_links['Category'] == category) &
                (df_validated_links['SubCategory'] == sub_cat)
            ][TEXT_COLUMN_NAME]
            
            # Take a random sample of up to 3 examples
            sample_size = min(len(examples), 3)
            if sample_size > 0:
                print(f"\n▶ Sub-Category: '{sub_cat}'")
                for i, verbatim in enumerate(examples.sample(sample_size, random_state=42)):
                    print(f"  {i+1}. \"{verbatim}\"")
        
        # --- ✨ UPDATED: PLOTTING SECTION (NOW WITH PERCENTAGES) ---
        for sentiment in ['Negative', 'Positive', 'Neutral']:
            plot_data = category_view[sentiment][category_view[sentiment] > 0].sort_values(ascending=True)
            
            if not plot_data.empty:
                # Convert counts to percentage of total
                plot_data_pct = plot_data / total_verbatims
                
                plt.figure(figsize=(10, len(plot_data) * 0.5 + 2))
                ax = plot_data_pct.plot(kind='barh', color={'Negative':'salmon', 'Positive':'mediumseagreen', 'Neutral':'silver'}[sentiment], width=0.9)
                
                # Format the x-axis as percentages
                ax.xaxis.set_major_formatter(mtick.PercentFormatter(1.0))

                # --- ADDED: Loop to add data call outs ---
                for i, value in enumerate(plot_data_pct):
                    # ax.text() places text on the axes
                    # x-coordinate: value (end of the bar) + a small offset
                    # y-coordinate: i (the position of the bar, 0, 1, 2...)
                    # text: formatted as a percentage string (e.g., '5.2%')
                    ax.text(value + (plot_data_pct.max() * 0.01), i, f'{value:.1%}', va='center')

                plt.title(f"{sentiment} Feedback for:\n'{category}'", fontsize=14)
                plt.xlabel("Percentage of Total Verbatims (%)", fontsize=12)
                plt.ylabel("")

                # ADDED: Adjust x-axis limit to make space for the labels
                ax.set_xlim(right=plot_data_pct.max() * 1.18)

                plt.tight_layout()
                
                s_cat = sanitize_filename(category)
                s_sent = sanitize_filename(sentiment)
                filename = f"granular_{s_cat}_{s_sent}.png"
                                
                fig = plt.gcf()
                fig.patch.set_edgecolor('grey')
                fig.patch.set_linewidth(1)

                plt.savefig(os.path.join(OUTPUT_FOLDER_PATH, filename))
                plt.show()
else:
    print("No valid Category/Sub-Category links found to generate a granular view.")

# Cell 8: Similarity Analysis (Jaccard & Semantic)
**This cell performs the deeper analysis on vocabulary overlap and semantic relationships.**

In [ ]:
# --- 1. Jaccard Similarity Between Sentiments ---
def get_top_ngrams(tokens, n, N_top):
    """Extracts the top N n-grams from a list of tokens."""
    ngrams_list = tokens if n == 1 else list(nltk.ngrams(tokens, n))
    return [item for item, freq in FreqDist(ngrams_list).most_common(N_top)]

def jaccard_similarity(list1, list2):
    """Calculates Jaccard similarity between two lists."""
    set1, set2 = set(list1), set(list2)
    intersection_len = len(set1.intersection(set2))
    union_len = len(set1.union(set2))
    return intersection_len / union_len if union_len > 0 else 0.0

# Calculate Jaccard similarity for top words and bigrams between sentiments
sentiments = ['Positive', 'Negative', 'Neutral']
tokens_by_sentiment = {s: word_tokenize(" ".join(df[df.sentiment_label == s]['cleaned_text'])) for s in sentiments}
comparisons = [("Positive", "Negative"), ("Positive", "Neutral"), ("Negative", "Neutral")]

results = {
    "Comparison": [f"{s1} vs. {s2}" for s1, s2 in comparisons],
    "Words (Top 20)": [jaccard_similarity(get_top_ngrams(tokens_by_sentiment[s1], 1, 20), get_top_ngrams(tokens_by_sentiment[s2], 1, 20)) for s1, s2 in comparisons],
    "Bigrams (Top 10)": [jaccard_similarity(get_top_ngrams(tokens_by_sentiment[s1], 2, 10), get_top_ngrams(tokens_by_sentiment[s2], 2, 10)) for s1, s2 in comparisons],
    "Trigrams (Top 10)": [jaccard_similarity(get_top_ngrams(tokens_by_sentiment[s1], 3, 10), get_top_ngrams(tokens_by_sentiment[s2], 3, 10)) for s1, s2 in comparisons],
}
similarity_df = pd.DataFrame(results)

# --- 2. Inter-Item Semantic & Jaccard Similarity ---
nlp = spacy.load("en_core_web_md")
top_words_overall = get_top_ngrams(all_tokens, 1, 20) # Top 20 Words
top_bigrams_overall = get_top_ngrams(all_tokens, 2, 10) # Top 10 Bigrams
top_trigrams_overall = get_top_ngrams(all_tokens, 3, 10) # Top 10 Trigrams

# Word vs Word (Semantic Similarity)
matrix_words = np.array([[nlp(w1).similarity(nlp(w2)) for w2 in top_words_overall] for w1 in top_words_overall])
similarity_df_words = pd.DataFrame(matrix_words, index=top_words_overall, columns=top_words_overall)

# Words vs Bigrams (Jaccard Similarity)
matrix_words_vs_bigrams = np.array([[jaccard_similarity([word], list(bigram)) for bigram in top_bigrams_overall] for word in top_words_overall])
similarity_df_words_vs_bigrams = pd.DataFrame(matrix_words_vs_bigrams, index=top_words_overall, columns=[' '.join(g) for g in top_bigrams_overall])

# Words vs Trigrams (Jaccard Similarity)
matrix_words_vs_trigrams = np.array([[jaccard_similarity([word], list(trigram)) for trigram in top_trigrams_overall] for word in top_words_overall])
similarity_df_words_vs_trigrams = pd.DataFrame(matrix_words_vs_trigrams, index=top_words_overall, columns=[' '.join(g) for g in top_trigrams_overall])

# Bigram vs Bigram (Jaccard Similarity)
matrix_bg = np.array([[jaccard_similarity(list(g1), list(g2)) for g2 in top_bigrams_overall] for g1 in top_bigrams_overall])
similarity_df_bigrams = pd.DataFrame(matrix_bg, index=[' '.join(g) for g in top_bigrams_overall], columns=[' '.join(g) for g in top_bigrams_overall])

# Trigram vs Trigram (Jaccard Similarity)
matrix_tg = np.array([[jaccard_similarity(list(g1), list(g2)) for g2 in top_trigrams_overall] for g1 in top_trigrams_overall])
similarity_df_trigrams = pd.DataFrame(matrix_tg, index=[' '.join(g) for g in top_trigrams_overall], columns=[' '.join(g) for g in top_trigrams_overall])


# --- 3. Generate and Save Heatmap Images ---
def create_and_save_heatmap(df_plot, title, filename, annot=False, cmap='viridis', figsize=(12, 10)):
    """Creates, displays, and saves a heatmap from a DataFrame."""
    plt.figure(figsize=figsize)
    sns.heatmap(df_plot, annot=annot, cmap=cmap, fmt=".2f")
    plt.title(title, fontsize=16)
        
    fig = plt.gcf()
    fig.patch.set_edgecolor('grey')
    fig.patch.set_linewidth(1)

    plt.savefig(os.path.join(OUTPUT_FOLDER_PATH, filename), bbox_inches='tight')
    plt.show()

# Generate and save the heatmaps
create_and_save_heatmap(similarity_df_words, 'Semantic Similarity of Top 20 Words', 'heatmap_words.png')
create_and_save_heatmap(similarity_df_bigrams, 'Jaccard Similarity of Top 10 Bigrams', 'heatmap_bigrams.png', annot=True, cmap='coolwarm')

# ADDED: Heatmap for Trigrams vs Trigrams
create_and_save_heatmap(similarity_df_trigrams, 'Jaccard Similarity of Top 10 Trigrams', 'heatmap_trigrams.png', annot=True, cmap='coolwarm')

# ADDED: Heatmap for Words vs Bigrams
create_and_save_heatmap(similarity_df_words_vs_bigrams, 'Jaccard Similarity: Top Words vs. Top Bigrams', 'heatmap_words_vs_bigrams.png', annot=True, cmap='magma', figsize=(10, 12))

# ADDED: Heatmap for Words vs Trigrams
create_and_save_heatmap(similarity_df_words_vs_trigrams, 'Jaccard Similarity: Top Words vs. Top Trigrams', 'heatmap_words_vs_trigrams.png', annot=True, cmap='magma', figsize=(10, 12))

# Cell 8.5: Categories & Sub-categories to Columns

In [ ]:
# --- One-Hot Encode Categories for Easy Filtering ---
print("--- ✨ Creating One-Hot Encoded Columns for Categories & Sub-Categories ---")

# Step 1: Get a unique list of all possible categories and sub-categories from your config
all_categories = list(CATEGORIES.keys())
all_subcategories = [subcat for sublist in CATEGORIES.values() for subcat in sublist]

# Step 2: Create a new column for each CATEGORY
# For each category, check if the string exists in the 'Category' column.
# .str.contains() returns True/False, which we convert to 1/0 with .astype(int).
# We use re.escape to handle any special characters in your category names safely.
print("Encoding main categories...")
for cat in tqdm(all_categories, desc="Encoding Categories"):
    # Ensure the column doesn't already exist to prevent errors on re-runs
    if cat not in df.columns:
        df[cat] = df['Category'].str.contains(re.escape(cat), na=False).astype(int)

# Step 3: Create a new column for each SUB-CATEGORY
print("Encoding sub-categories...")
for subcat in tqdm(all_subcategories, desc="Encoding Sub-Categories"):
     # Ensure the column doesn't already exist
    if subcat not in df.columns:
        df[subcat] = df['Sub-Category'].str.contains(re.escape(subcat), na=False).astype(int)

# Step 4: Display a preview of the new columns
print("\n✅ One-hot encoding complete.")
print("--- Preview of DataFrame with New Encoded Columns ---")
# Create a list of columns to show: original text, categories, and the first few new columns
preview_cols = [TEXT_COLUMN_NAME, 'Category', 'Sub-Category'] + all_categories[:2] + all_subcategories[:2]
display(df[preview_cols].head())

# --- Run Significance Testing (T-tests) ---
SCORE_COLUMN_NAME = 'CSAT Score'

print(f"--- 🏃 Running Significance Tests against '{SCORE_COLUMN_NAME}' column ---")

# --- Safety Check ---
if SCORE_COLUMN_NAME not in df.columns:
    print("="*80)
    print(f"❌ FATAL ERROR: The score column '{SCORE_COLUMN_NAME}' was not found in the DataFrame.")
    print("   Please update the 'SCORE_COLUMN_NAME' variable in this cell and run it again.")
    print("="*80)
else:
    # ✨ Step 1: Create a mapping of each sub-category to its parent category
    all_categories = list(CATEGORIES.keys())
    subcat_to_cat_map = {
        subcat: cat 
        for cat, sublist in CATEGORIES.items() 
        for subcat in sublist
    }

    # ✨ Step 2: Run tests for Categories ONLY
    category_results = []
    for cat_col in tqdm(all_categories, desc="Testing Level 1 Categories"):
        if cat_col in df.columns:
            group_present = df[df[cat_col] == 1][SCORE_COLUMN_NAME].dropna()
            group_absent = df[df[cat_col] == 0][SCORE_COLUMN_NAME].dropna()
            if len(group_present) > 1 and len(group_absent) > 1:
                t_stat, p_value = stats.ttest_ind(group_present, group_absent, equal_var=False)
                category_results.append({
                    'Category': cat_col,
                    'Mean Score When Present': group_present.mean(),
                    'Mean Score When Absent': group_absent.mean(),
                    'p-value': p_value,
                    'Comment Count': len(group_present)
                })

    # ✨ Step 3: Run tests for Sub-Categories ONLY
    sub_category_results = []
    for subcat_col, parent_cat in tqdm(subcat_to_cat_map.items(), desc="Testing Level 2 Sub-Categories"):
        if subcat_col in df.columns:
            group_present = df[df[subcat_col] == 1][SCORE_COLUMN_NAME].dropna()
            group_absent = df[df[subcat_col] == 0][SCORE_COLUMN_NAME].dropna()
            if len(group_present) > 1 and len(group_absent) > 1:
                t_stat, p_value = stats.ttest_ind(group_present, group_absent, equal_var=False)
                sub_category_results.append({
                    'Parent Category': parent_cat,
                    'Sub-Category': subcat_col,
                    'Mean Score When Present': group_present.mean(),
                    'Mean Score When Absent': group_absent.mean(),
                    'p-value': p_value,
                    'Comment Count': len(group_present)
                })

    # --- ✨ Step 4: Assemble the three final tables ---

    # Table 1: Category-Only Results
    df_significance_categories = pd.DataFrame(category_results)
    if not df_significance_categories.empty:
        df_significance_categories['Significant (p < 0.05)'] = df_significance_categories['p-value'] < 0.05
        df_significance_categories = df_significance_categories.sort_values(by='p-value', ascending=True)

    # Table 2: Sub-Category-Only Results
    df_significance_subcategories = pd.DataFrame(sub_category_results)
    if not df_significance_subcategories.empty:
        df_significance_subcategories['Significant (p < 0.05)'] = df_significance_subcategories['p-value'] < 0.05
        df_significance_subcategories = df_significance_subcategories.sort_values(by='p-value', ascending=True)

    # Table 3: Overall Combined Results
    df_overall_cat = df_significance_categories.copy()
    df_overall_cat['Driver'] = 'Level 1: ' + df_overall_cat['Category']
    
    df_overall_sub = df_significance_subcategories.copy()
    df_overall_sub['Driver'] = 'Level 2: ' + df_overall_sub['Sub-Category']
    
    df_significance_overall = pd.concat([
        df_overall_cat.drop(columns=['Category']),
        df_overall_sub.drop(columns=['Parent Category', 'Sub-Category'])
    ])
    if not df_significance_overall.empty:
        # Reorder columns to put Driver first
        cols = ['Driver'] + [col for col in df_significance_overall.columns if col != 'Driver']
        df_significance_overall = df_significance_overall[cols]
        df_significance_overall = df_significance_overall.sort_values(by='p-value', ascending=True)

    # --- ✨ Step 5: Display all three tables ---
    print("\n" + "="*50)
    print("      SUMMARY OF SIGNIFICANCE TEST RESULTS")
    print("="*50 + "\n")

    print("\n--- 📜 Table 1: Overall Significance (Combined) ---")
    display(df_significance_overall.round(3))
    
    print("\n--- 📂 Table 2: Category-Only Significance ---")
    display(df_significance_categories.round(3))

    print("\n--- 📄 Table 3: Sub-Category Significance (with Parent) ---")
    display(df_significance_subcategories.round(3))

    # This ensures the next cell (for plotting) uses the main combined table
    df_significance = df_significance_overall

# Cell 8.6: Impact-Significance Plot

In [ ]:
# ✨ Import the new library
try:
    from adjustText import adjust_text
except ImportError:
    !pip install adjusttext
    from adjustText import adjust_text

# --- 10. Create Separate Impact-Significance Matrix Plots ---

# ✨ Step 1: Define a reusable function to create the plot
def create_impact_plot(df_data, name_column, title, filename):
    """
    Generates and saves an impact-significance scatter plot with auto-adjusting labels.
    """
    print(f"--- 📊 Generating Plot: {title} ---")
    
    if df_data is None or df_data.empty:
        print(f"⚠️ Skipping plot '{title}': Input DataFrame is empty or does not exist.")
        return

    df_plot_data = df_data.copy()
    df_plot_data['Score_Difference'] = df_plot_data['Mean Score When Present'] - df_plot_data['Mean Score When Absent']
    
    bins = [0, 0.05, 0.1, 1.01]
    labels = ['p < 0.05 (Significant)', '0.05 <= p < 0.1', 'p >= 0.1 (Not Significant)']
    df_plot_data['Significance Level'] = pd.cut(df_plot_data['p-value'], bins=bins, labels=labels, right=False)

    points_to_label = df_plot_data.sort_values(name_column).copy()
    points_to_label['Label_ID'] = range(1, len(points_to_label) + 1)
    
    plt.figure(figsize=(18, 12))
    
    # ✨ CHANGE #1: Add legend='brief' to simplify the size legend
    ax = sns.scatterplot(
        data=df_plot_data,
        x='Score_Difference',
        y='p-value',
        hue='Significance Level',
        palette={
            'p < 0.05 (Significant)': 'springgreen',
            '0.05 <= p < 0.1': 'orange',
            'p >= 0.1 (Not Significant)': 'grey'
        },
        size='Comment Count',
        sizes=(50, 500),
        alpha=0.7,
        legend='brief' 
    )

    plt.axvline(x=0, color='black', linestyle='--', linewidth=1)
    plt.axhline(y=0.05, color='green', linestyle='--', linewidth=1)
    plt.axhline(y=0.10, color='darkorange', linestyle='--', linewidth=1)
    plt.title(title, fontsize=16)
    plt.xlabel("Impact on Score (Mean Score Difference)", fontsize=12)
    plt.ylabel("p-value (Statistical Significance)", fontsize=12)
    plt.ylim(-0.05, max(1.0, df_plot_data['p-value'].max() * 1.05))

    texts = []
    for i, row in points_to_label.iterrows():
        texts.append(
            plt.text(
                x=row['Score_Difference'],
                y=row['p-value'],
                s=f" {row['Label_ID']}",
                fontdict=dict(color='black', weight='bold', size=8)
            )
        )
        
    plt.text(0.01, 0.05, ' p=0.05 (Significance Threshold)', color='green', ha='left', va='bottom', fontsize=10, style='italic')
    plt.text(0.01, 0.10, ' p=0.10 (Significance at 90% Confidence Threshold)', color='darkorange', ha='left', va='bottom', fontsize=10, style='italic')
    
    if not points_to_label.empty:
        legend_text = "Labels:\n" + "\n".join(
            [f"{row['Label_ID']}: {row[name_column]}" for i, row in points_to_label.iterrows()]
        )
        
        # Position the main color and size legends
        plt.legend(title='Legend', bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
        
        # ✨ CHANGE #2: Move the custom text box further down (y=0.7)
        plt.text(1.02, 0.70, legend_text, transform=ax.transAxes, fontsize=9,
                 verticalalignment='top', bbox=dict(boxstyle='round,pad=0.5', fc='aliceblue', alpha=0.9))

    adjust_text(texts, arrowprops=dict(arrowstyle='->', color='black', lw=0.5))

    plt.subplots_adjust(right=0.7) 
    fig = plt.gcf()
    fig.patch.set_edgecolor('grey')
    fig.patch.set_linewidth(1)
    
    plt.savefig(os.path.join(OUTPUT_FOLDER_PATH, filename), bbox_inches='tight')
    print(f"✅ Plot saved as '{filename}'")
    plt.show()

# ✨ Step 2: Call the function twice, once for categories and once for sub-categories
# (This part of the code remains the same)
create_impact_plot(
    df_data=df_significance_categories,
    name_column='Category',
    title=f"Impact vs. Significance Matrix for Categories",
    filename='impact_matrix_categories.png'
)

create_impact_plot(
    df_data=df_significance_subcategories,
    name_column='Sub-Category',
    title=f"Impact vs. Significance Matrix for Sub-Categories",
    filename='impact_matrix_subcategories.png'
)

# Cell 9: Final Report Generation
**This final cell gathers every DataFrame and image and compiles them into a single, multi-sheet Excel report.**

In [ ]:
# --- Helper Functions ---
def sanitize_filename(name):
    name = re.sub(r'[^\w\s-]', '', name).strip().lower()
    name = re.sub(r'[-\s]+', '_', name)
    return name

def add_image_if_exists(worksheet, image_filename, cell_anchor, folder_path):
    image_path = os.path.join(folder_path, image_filename)
    if os.path.exists(image_path):
        img = Image(image_path)
        worksheet.add_image(img, cell_anchor)
    else:
        print(f"Warning: Image file not found at {image_path}, skipping.")

def apply_borders(ws, start_row, end_row, start_col, end_col):
    """Applies a thin border to all cells in a given range."""
    thin_border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))
    for row in ws.iter_rows(min_row=start_row, max_row=end_row, min_col=start_col, max_col=end_col):
        for cell in row:
            cell.border = thin_border

def write_df_with_borders(writer, sheet_name, df, start_row=0, **kwargs):
    """Writes a DataFrame to Excel and applies borders to all its cells."""
    df.to_excel(writer, sheet_name=sheet_name, startrow=start_row, **kwargs)
    ws = writer.sheets[sheet_name]
    index_cols = df.index.nlevels if kwargs.get('index', True) else 0
    end_col = index_cols + len(df.columns)
    apply_borders(ws, start_row + 1, start_row + 1 + len(df), 1, end_col)

# --- Versioned Filename Generation ---
current_date = datetime.now().strftime('%Y-%m-%d')
base_filename = f"{current_date}_{PROGRAM_NAME}_{KPIS_IN_SCOPE}_{LOBS_IN_SCOPE}_Verbatim_Analysis"
minor_version = 0
while True:
    version_str = f"v{MAJOR_VERSION:02d}.{minor_version:02d}"
    output_filename = f"{base_filename}_{version_str}.xlsx"
    full_path = os.path.join(OUTPUT_FOLDER_PATH, output_filename)
    if not os.path.exists(full_path):
        break
    minor_version += 1

# --- Use ExcelWriter to save all results ---
with pd.ExcelWriter(full_path, engine='openpyxl') as writer:
    print(f"\n--- 🚀 Writing to Excel file: {output_filename} ---")

    # --- Sheet 1: Executive Summary ---
    print("Writing Sheet: Executive_Summary")
    row_offset = 0
    write_df_with_borders(writer, 'Executive_Summary', df_summary_metrics, start_row=row_offset, index=False)
    pos1 = row_offset + len(df_summary_metrics) + 3
    write_df_with_borders(writer, 'Executive_Summary', df_topic_evaluation.round(3), start_row=pos1, index=False)
    pos2 = pos1 + len(df_topic_evaluation) + 3
    write_df_with_borders(writer, 'Executive_Summary', df_cat_counts.to_frame(name='Count'), start_row=pos2)
    pos3 = pos2 + len(df_cat_counts) + 3
    write_df_with_borders(writer, 'Executive_Summary', df_subcat_counts.to_frame(name='Count'), start_row=pos3)
    ws_summary = writer.sheets['Executive_Summary']
    ws_summary.cell(row=row_offset + 1, column=1, value="Categorization Summary")
    ws_summary.cell(row=pos1 + 1, column=1, value="Topic Model Evaluation (Silhouette & Coherence)")
    ws_summary.cell(row=pos2 + 1, column=1, value="Overall Category Counts")
    ws_summary.cell(row=pos3 + 1, column=1, value="Overall Sub-Category Counts")
    add_image_if_exists(ws_summary, 'topic_evaluation_charts.png', 'E2', OUTPUT_FOLDER_PATH)
    add_image_if_exists(ws_summary, 'freq_cat_overall.png', f'E{pos2 + 2}', OUTPUT_FOLDER_PATH)
    add_image_if_exists(ws_summary, 'freq_subcat_overall.png', f'E{pos3 + 2}', OUTPUT_FOLDER_PATH)

    # --- Sheet 2: Category Deep-Dive ---
    sheet_name = 'Category_Deep_Dive'
    print(f"Writing Sheet: {sheet_name}")
    if 'granular_sentiment_df' in locals() and not granular_sentiment_df.empty:
        current_row = 1
        unique_categories = granular_sentiment_df.index.get_level_values('Category').unique()

        for category in unique_categories:
            category_view = granular_sentiment_df.loc[category].sort_values(by='Total', ascending=False)
            category_view = category_view[['Negative', 'Neutral', 'Positive', 'Total']]
            
            examples_list = []
            for sub_cat in category_view.index:
                examples = df_validated_links[(df_validated_links['Category'] == category) & (df_validated_links['SubCategory'] == sub_cat)][TEXT_COLUMN_NAME]
                sample_size = min(len(examples), 3)
                if sample_size > 0:
                    formatted_examples = "\n".join([f"{j+1}. \"{v}\"" for j, v in enumerate(examples.sample(sample_size, random_state=42))])
                    examples_list.append(formatted_examples)
                else:
                    examples_list.append("No examples found.")
            category_view['Verbatim Examples'] = examples_list
            
            write_df_with_borders(writer, sheet_name, category_view, start_row=current_row)
            
            ws_deep_dive = writer.sheets[sheet_name]
            ws_deep_dive.cell(row=current_row, column=1, value=f"Deep-Dive for Category: {category}")
            
            # Format Column A (Sub-Category)
            ws_deep_dive.column_dimensions['A'].width = 30
            for r in range(current_row + 2, current_row + 2 + len(category_view)):
                ws_deep_dive[f'A{r}'].alignment = Alignment(wrap_text=True, vertical='top')

            # Format Column G (Verbatim Examples)
            ws_deep_dive.column_dimensions['F'].width = 90
            for r in range(current_row + 2, current_row + 2 + len(category_view)):
                ws_deep_dive[f'G{r}'].alignment = Alignment(wrap_text=True, vertical='top')

            s_cat = sanitize_filename(category)
            add_image_if_exists(ws_deep_dive, f"granular_{s_cat}_negative.png", f'K{current_row + 2}', OUTPUT_FOLDER_PATH)
            add_image_if_exists(ws_deep_dive, f"granular_{s_cat}_positive.png", f'AB{current_row + 2}', OUTPUT_FOLDER_PATH)
            add_image_if_exists(ws_deep_dive, f"granular_{s_cat}_neutral.png", f'AS{current_row + 2}', OUTPUT_FOLDER_PATH)

            current_row += len(category_view) + 30
    else:
        pd.DataFrame([{'Message': "No granular data available for this report."}]).to_excel(writer, sheet_name=sheet_name, index=False)

    # --- Sheet 3: Sentiment Analysis ---
    print("Writing Sheet: Sentiment_Analysis")
    row_offset = 0
    write_df_with_borders(writer, 'Sentiment_Analysis', df_sentiment_by_cat, start_row=row_offset)
    pos1 = row_offset + len(df_sentiment_by_cat) + 3
    write_df_with_borders(writer, 'Sentiment_Analysis', df_sentiment_by_subcat, start_row=pos1)
    ws_sentiment = writer.sheets['Sentiment_Analysis']
    ws_sentiment.cell(row=row_offset + 1, column=1, value="Sentiment Breakdown by Category")
    ws_sentiment.cell(row=pos1 + 1, column=1, value="Sentiment Breakdown by Sub-Category")
    add_image_if_exists(ws_sentiment, 'sentiment_distribution.png', 'G2', OUTPUT_FOLDER_PATH)
    charts_start_row = pos1 + len(df_sentiment_by_subcat) + 5
    sentiment_charts = [('Positive Sentiment', 'freq_cat_positive.png', 'freq_subcat_positive.png'), ('Negative Sentiment', 'freq_cat_negative.png', 'freq_subcat_negative.png'), ('Neutral Sentiment', 'freq_cat_neutral.png', 'freq_subcat_neutral.png')]
    current_row = charts_start_row
    for title, cat_chart, subcat_chart in sentiment_charts:
        ws_sentiment.cell(row=current_row, column=1, value=title)
        add_image_if_exists(ws_sentiment, cat_chart, f'A{current_row + 1}', OUTPUT_FOLDER_PATH)
        add_image_if_exists(ws_sentiment, subcat_chart, f'K{current_row + 1}', OUTPUT_FOLDER_PATH)
        current_row += 40
    
    # --- Sheet 4: Exploratory Analysis ---
    print("Writing Sheet: Exploratory_Analysis")
    row_offset = 0
    write_df_with_borders(writer, 'Exploratory_Analysis', df_top_words, start_row=row_offset, index=False)
    pos1 = row_offset + len(df_top_words) + 3
    write_df_with_borders(writer, 'Exploratory_Analysis', df_top_bigrams, start_row=pos1, index=False)
    pos2 = pos1 + len(df_top_bigrams) + 3
    write_df_with_borders(writer, 'Exploratory_Analysis', df_top_trigrams, start_row=pos2, index=False)
    ws_exploratory = writer.sheets['Exploratory_Analysis']
    ws_exploratory.cell(row=row_offset + 1, column=1, value="Top 20 Words")
    ws_exploratory.cell(row=pos1 + 1, column=1, value="Top 10 Bigrams")
    ws_exploratory.cell(row=pos2 + 1, column=1, value="Top 10 Trigrams")
    add_image_if_exists(ws_exploratory, 'wordcloud_all.png', 'E2', OUTPUT_FOLDER_PATH)
    add_image_if_exists(ws_exploratory, 'wordcloud_positive.png', 'E25', OUTPUT_FOLDER_PATH)
    add_image_if_exists(ws_exploratory, 'wordcloud_negative.png', 'E50', OUTPUT_FOLDER_PATH)

    # --- Sheet 5: Topic Modeling & Uncategorized ---
    print("Writing Sheet: Topic_Modeling_Deep_Dive")
    row_offset = 0
    write_df_with_borders(writer, 'Topic_Modeling_Deep_Dive', df_topics, start_row=row_offset, index=False)
    pos1 = row_offset + len(df_topics) + 3
    write_df_with_borders(writer, 'Topic_Modeling_Deep_Dive', df_uncategorized_keywords, start_row=pos1, index=False)
    ws_topics = writer.sheets['Topic_Modeling_Deep_Dive']
    ws_topics.cell(row=row_offset + 1, column=1, value="Discovered Topics (LDA)")
    ws_topics.cell(row=pos1 + 1, column=1, value="Top Keywords in Uncategorized Verbatims")

    # --- Sheet 6: Similarity Analysis ---
    print("Writing Sheet: Similarity_Analysis")
    write_df_with_borders(writer, 'Similarity_Analysis', similarity_df, index=False)
    ws_similarity = writer.sheets['Similarity_Analysis']
    ws_similarity.cell(row=1, column=1, value="Jaccard Similarity Between Sentiments")
    current_row = len(similarity_df) + 5
    ws_similarity.cell(row=current_row, column=1, value="Semantic Similarity of Top 20 Words")
    add_image_if_exists(ws_similarity, 'heatmap_words.png', f'A{current_row + 1}', OUTPUT_FOLDER_PATH)
    ws_similarity.cell(row=current_row, column=15, value="Jaccard Similarity of Top 10 Bigrams")
    add_image_if_exists(ws_similarity, 'heatmap_bigrams.png', f'O{current_row + 1}', OUTPUT_FOLDER_PATH)
    current_row += 55
    ws_similarity.cell(row=current_row, column=1, value="Jaccard Similarity of Top 10 Trigrams")
    add_image_if_exists(ws_similarity, 'heatmap_trigrams.png', f'A{current_row + 1}', OUTPUT_FOLDER_PATH)
    ws_similarity.cell(row=current_row, column=15, value="Jaccard Similarity: Top Words vs. Top Bigrams")
    add_image_if_exists(ws_similarity, 'heatmap_words_vs_bigrams.png', f'O{current_row + 1}', OUTPUT_FOLDER_PATH)
    current_row += 55
    ws_similarity.cell(row=current_row, column=1, value="Jaccard Similarity: Top Words vs. Top Trigrams")
    add_image_if_exists(ws_similarity, 'heatmap_words_vs_trigrams.png', f'A{current_row + 1}', OUTPUT_FOLDER_PATH)

# --- Sheet 7: Score Driver Analysis (Overall, Categories, and Sub-Categories) ---
    sheet_name = 'Score_Driver_Analysis'
    print(f"Writing Sheet: {sheet_name}")

    # Check if the main DataFrame exists to proceed
    if 'df_significance_overall' in locals() and not df_significance_overall.empty:
        
        # --- Write Table 1: Overall Significance ---
        current_row = 0
        write_df_with_borders(writer, sheet_name, df_significance_overall.round(3), start_row=current_row, index=False)
        ws_drivers = writer.sheets[sheet_name]
        ws_drivers.cell(row=current_row + 1, column=1, value=f"Overall Key Driver Analysis for '{SCORE_COLUMN_NAME}'")

        # --- Write Table 2: Category-Only Significance ---
        # Calculate the starting row for the next table, adding some padding
        current_row += len(df_significance_overall) + 4 
        if 'df_significance_categories' in locals() and not df_significance_categories.empty:
            write_df_with_borders(writer, sheet_name, df_significance_categories.round(3), start_row=current_row, index=False)
            ws_drivers.cell(row=current_row + 1, column=1, value="Category-Only Significance")

            # Update current_row for the next table
            current_row += len(df_significance_categories) + 4

        # --- Write Table 3: Sub-Category-Only Significance ---
        if 'df_significance_subcategories' in locals() and not df_significance_subcategories.empty:
            write_df_with_borders(writer, sheet_name, df_significance_subcategories.round(3), start_row=current_row, index=False)
            ws_drivers.cell(row=current_row + 1, column=1, value="Sub-Category-Only Significance")

        # --- Add Plots and Adjust Column Widths ---
        add_image_if_exists(ws_drivers, 'impact_matrix_categories.png', 'J2', OUTPUT_FOLDER_PATH)
        add_image_if_exists(ws_drivers, 'impact_matrix_subcategories.png', 'J60', OUTPUT_FOLDER_PATH)
        
        # Auto-adjust column widths for better readability
        for col in ws_drivers.columns:
            max_length = 0
            column = col[0].column_letter
            for cell in col:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(cell.value)
                except:
                    pass
            adjusted_width = (max_length + 2)
            ws_drivers.column_dimensions[column].width = adjusted_width  
    
    # --- Sheet 8: Categorization Results (Full Data) ---
    print("Writing Sheet: Categorization_Results")
    write_df_with_borders(writer, 'Categorization_Results', df, index=False)

print(f"\n✅ All analysis results and plots have been saved to a multi-sheet file:")
print(f"   {full_path}")

## Clear the /Kaggle/Working output directory

In [ ]:
# # Ctrl + / to un/comment out code while highlighted

# import os
# import shutil

# # This is the directory you want to clear
# output_dir = '/kaggle/working/'

# # Loop through everything in the directory
# for filename in os.listdir(output_dir):
#     file_path = os.path.join(output_dir, filename)
#     try:
#         # If it's a file or link, delete it
#         if os.path.isfile(file_path) or os.path.islink(file_path):
#             os.unlink(file_path)
#         # If it's a directory, delete it and all its contents
#         elif os.path.isdir(file_path):
#             shutil.rmtree(file_path)
#         print(f"Deleted: {filename}")
#     except Exception as e:
#         print(f'Failed to delete {file_path}. Reason: {e}')

# print("\n✅ Output directory has been cleared.")